In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pagets_Disease_of_bone/GSE7849'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Age-Specific Differences of Oncogenic Pathway Deregulation Seen in Human Breast Tumors"
!Series_summary	"Breast cancer arising in young women has a poorer prognosis, is less likely to be hormone sensitive, and represents a particularly challenging clinical entity.  The biology driving the aggressive nature of breast cancer arising in young women has yet to be defined."
!Series_summary	"Among 784 patients with early stage breast cancer, using prospectively-defined, age-specific cohorts (young <= 45 years; older >= 65 years), 411 eligible patients (n = 200 < 45 years; n = 211 >= 65 years) with clinically-annotated Affymetrix microarray data were identified. Gene set enrichment analyses, signatures of oncogenic pathway deregulation and predictors of chemotherapy sensitivity were evaluated within the two age-defined cohorts."
!Series_summary	"In comparing deregulation of oncogenic pathways between age groups, a statistically higher probability of PI3K

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking gene expression data availability
# Assuming the matrix file contains gene expression data if the filenames suggest Affymetrix microarray data
is_gene_available = True  

# Variable Availability and Data Type Conversion
def convert_trait(value):
    return None  # Trait data not available

# Age conversion function
def convert_age(value):
    try:
        return float(value.split('=')[-1].strip())
    except ValueError:
        return None

# Gender conversion function
def convert_gender(value):
    gender_map = {'female': 0, 'male': 1}
    gender = value.split('=')[-1].strip().lower()
    return gender_map.get(gender, None)

# Setting the appropriate rows for each variable based on the provided data.
# Checking keys for 'Pagets_Disease_of_bone'
# There is no explicit mention of Paget's Disease of Bone, assuming 4 for histological descriptions but we'll consider it unavailable.

# Checking keys for 'age'
for key, values in clinical_data.items():
    if 'Age_at_Dx' in values[0]:  # Example key string
        age_row = key
        break
# If no proper key is found, we leave age_row as None

# Checking keys for 'gender'
for key, values in clinical_data.items():
    # gender data not provided
    gender_row = None

# Metadata saving
save_cohort_info('GSE7849', './preprocessed/Pagets_Disease_of_bone/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pagets_Disease_of_bone', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pagets_Disease_of_bone/trait_data/GSE7849.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Pagets_Disease_of_bone/cohort_info.json
